<a href="https://colab.research.google.com/github/gugaio/transformer/blob/master/Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [3]:
from datasets import load_dataset

ds = load_dataset("codeparrot/github-code", streaming=True, split="train")


In [4]:
i = 0
text = ""
for example in ds:
    text += example["code"]
    i+=1
    if i > 70:
      break

In [5]:
print(text[:30])

'use strict';

var clear      


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

124

In [7]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [ stoi[c] for c in s]
decode = lambda tokens: ''.join([ itos[t]  for t in tokens])

In [8]:
encoded = encode("use strict")
encoded

[88, 86, 72, 3, 86, 87, 85, 76, 70, 87]

In [9]:
decode(encoded)

'use strict'

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
data.size()

torch.Size([455811])

In [11]:
n_train = int(len(text)*0.9)
n_train

410229

In [12]:
train_data = data[:n_train]
eval_data = data[n_train:]

In [13]:
batch_size = 4
block_size = 8
train_data[:block_size+1]

tensor([10, 88, 86, 72,  3, 86, 87, 85, 76])

In [14]:
for t in range(block_size):
  input = train_data[:t+1]
  target = train_data[t]
  print(f"when input is {input} target is {target}")

when input is tensor([10]) target is 10
when input is tensor([10, 88]) target is 88
when input is tensor([10, 88, 86]) target is 86
when input is tensor([10, 88, 86, 72]) target is 72
when input is tensor([10, 88, 86, 72,  3]) target is 3
when input is tensor([10, 88, 86, 72,  3, 86]) target is 86
when input is tensor([10, 88, 86, 72,  3, 86, 87]) target is 87
when input is tensor([10, 88, 86, 72,  3, 86, 87, 85]) target is 85


In [15]:
torch.randint( len(train_data) - block_size, (batch_size,))

tensor([151212, 280129, 165411, 315275])

In [16]:
def batch(split):
  data = train_data if split == 'train' else eval_data
  ix = torch.randint( len(data) - block_size, (batch_size,))
  batch_inputs = [data[i:i+block_size] for i in ix]
  batch_targets = [data[i+1:i+block_size+1] for i in ix]
  x = torch.stack(batch_inputs)
  y = torch.stack(batch_targets)
  return x,y

xb, yb = batch('train')
print(f"input shape {xb.shape} and target shape {yb.shape}")

input shape torch.Size([4, 8]) and target shape torch.Size([4, 8])


In [17]:
for b in range(batch_size):
  for t in range(block_size):
    input = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {input.tolist()} target is {target}")

when input is [73] target is 76
when input is [73, 76] target is 81
when input is [73, 76, 81] target is 71
when input is [73, 76, 81, 71] target is 66
when input is [73, 76, 81, 71, 66] target is 89
when input is [73, 76, 81, 71, 66, 89] target is 71
when input is [73, 76, 81, 71, 66, 89, 71] target is 72
when input is [73, 76, 81, 71, 66, 89, 71, 72] target is 89
when input is [32] target is 3
when input is [32, 3] target is 19
when input is [32, 3, 19] target is 12
when input is [32, 3, 19, 12] target is 3
when input is [32, 3, 19, 12, 3] target is 94
when input is [32, 3, 19, 12, 3, 94] target is 1
when input is [32, 3, 19, 12, 3, 94, 1] target is 0
when input is [32, 3, 19, 12, 3, 94, 1, 0] target is 0
when input is [17] target is 5
when input is [17, 5] target is 12
when input is [17, 5, 12] target is 12
when input is [17, 5, 12, 12] target is 30
when input is [17, 5, 12, 12, 30] target is 1
when input is [17, 5, 12, 12, 30, 1] target is 0
when input is [17, 5, 12, 12, 30, 1, 0] 

In [18]:
class BigramModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, y=None):
    logits = self.embedding(x) # B,T => B,T,D
    if y is None:
      loss = None
    else:
      B,T,D = logits.shape
      logits = logits.view(B*T, D) # B,T,D => B*T, D
      y = y.view(-1) # B,T,D => B*T, D
      loss = F.cross_entropy(logits, y)
    return logits, loss

  def generate(self, x, max_new_tokens):
    print("Generating...")
    for _ in range(max_new_tokens):
      logits, loss = self(x)
      last_logits = logits[:,-1,:]
      probs = F.softmax(last_logits, dim=-1)
      x_next = torch.multinomial(probs, num_samples=1)
      x = torch.cat((x, x_next), dim=1)
    return x

model = BigramModel(vocab_size)
out, loss = model(xb, yb)
print(f"Output shape {out.shape}")
print(f"Batch Loss  {loss}")

sample = torch.zeros((1,1), dtype=torch.long)
gx=model.generate(x=sample, max_new_tokens=4)
print(decode(gx[0].tolist()))

Output shape torch.Size([32, 124])
Batch Loss  5.35922384262085
Generating...
	*a0ę


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 1000
for step in range(epochs):
  xb, yb = batch('train')
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(loss.item())

3.5978686809539795
3.069528818130493
3.0443880558013916
3.5828821659088135
2.8308074474334717
3.2708215713500977
3.463637351989746
3.4711921215057373
3.5938289165496826
3.451181173324585
3.0611648559570312
3.1055643558502197
3.7431232929229736
3.117169141769409
3.884014844894409
2.982602834701538
3.647014617919922
3.1441996097564697
3.2375612258911133
3.2238292694091797
3.204946994781494
3.4264323711395264
3.39215350151062
3.317739248275757
3.468334674835205
3.436131477355957
3.9249179363250732
3.9381275177001953
3.31693959236145
3.506251335144043
3.9822945594787598
3.311087131500244
3.6982309818267822
3.5542244911193848
3.996973752975464
3.4979681968688965
3.7003061771392822
3.598902940750122
3.296269178390503
3.499169111251831
3.3149313926696777
3.693540096282959
3.1371984481811523
3.2365047931671143
3.580733299255371
3.464724063873291
4.0254340171813965
3.6283082962036133
3.27730655670166
3.5641391277313232
3.805882453918457
3.4947240352630615
3.4391050338745117
3.7078235149383545
3